In [1]:
#Importing Libraries
import os 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization,Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

In [2]:
os.listdir('/content/drive/MyDrive/ML Club NITS/Datasets/natural_images')

['person', 'airplane', 'motorbike', 'fruit', 'flower', 'car', 'cat', 'dog']

In [3]:
def img_read(folder):
    imgg=[] 
    labels = []
    classes = ['person', 'airplane', 'motorbike', 'fruit', 'flower', 'car', 'cat', 'dog']
    for i in classes:
        for filename in tqdm(os.listdir(os.path.join(folder , i))):
            img = cv2.imread(os.path.join(os.path.join(folder,i) , filename)) #Reading the image file from the base directory
            img = cv2.resize(img , (150 , 150)) # Resizing the images to a matrix of (150,150)
            imgg.append(img) #Storing the Image one by one in an empty list
            labels.append(i) #Storing the labels of the image one by one in an empty list 
    return imgg , labels

In [4]:
img , label = img_read('/content/drive/MyDrive/ML Club NITS/Datasets/natural_images')

100%|██████████| 702/702 [00:08<00:00, 83.69it/s] 


In [6]:
#Creating data generator training and validation dataset
dir = '/content/drive/MyDrive/ML Club NITS/Datasets/natural_images'
data_gen = ImageDataGenerator(rescale = 1./255,
                              horizontal_flip=True,
                              brightness_range=[0.4,1.5],# values less then 1 darken the image and greater than 1 brighten.
                              validation_split=0.2)
train_datagen = data_gen.flow_from_directory(dir,
                                         batch_size = 64,
                                         class_mode = "categorical",
                                         target_size = (150,150),
                                         subset = "training")
val_datagen = data_gen.flow_from_directory(
    dir,
    batch_size =64 ,
    class_mode = "categorical",
    target_size = (150,150),
    subset = "validation"
)

Found 5522 images belonging to 8 classes.
Found 1377 images belonging to 8 classes.


In [7]:
#Model Construction 
inputs = tf.keras.Input(shape=(150,150,3))
x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(32, (3, 3), activation='relu', padding='same',)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.2)(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same',)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.2)(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)
x = Dense(16,activation='relu')(x)
output = Dense(8, activation='softmax')(x)
model = Model(inputs, output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
#Callbacks
EP = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5
)
MCP = tf.keras.callbacks.ModelCheckpoint(
    filepath = './weights' ,
    monitor='accuracy' , 
    save_best_only=True
)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 150, 150, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 32)      18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 75, 75, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 32)        9248      
                                                             

In [10]:
history = model.fit(
    train_datagen,
    batch_size=128,
    epochs=5,
    #steps_per_epoch=100,
    verbose=1,
    validation_data=val_datagen,
    callbacks=[EP,MCP]
)

Epoch 1/5
87/87 [==============================] - ETA: 0s - loss: 1.2737 - accuracy: 0.5522

87/87 [==============================] - 40s 346ms/step - loss: 1.2737 - accuracy: 0.5522 - val_loss: 1.0926 - val_accuracy: 0.6652
Epoch 2/5
87/87 [==============================] - ETA: 0s - loss: 0.6563 - accuracy: 0.7807

87/87 [==============================] - 28s 327ms/step - loss: 0.6563 - accuracy: 0.7807 - val_loss: 0.5617 - val_accuracy: 0.7996
Epoch 3/5
87/87 [==============================] - ETA: 0s - loss: 0.4276 - accuracy: 0.8519

87/87 [==============================] - 28s 325ms/step - loss: 0.4276 - accuracy: 0.8519 - val_loss: 0.4869 - val_accuracy: 0.8301
Epoch 4/5
87/87 [==============================] - ETA: 0s - loss: 0.3539 - accuracy: 0.8740

87/87 [==============================] - 28s 325ms/step - loss: 0.3539 - accuracy: 0.8740 - val_loss: 0.4060 - val_accuracy: 0.8642
Epoch 5/5
87/87 [==============================] - ETA: 0s - loss: 0.3171 - accuracy: 0.8874

87/87 [==============================] - 28s 325ms/step - loss: 0.3171 - accuracy: 0.8874 - val_loss: 0.4168 - val_accuracy: 0.8577


In [12]:
model.save("model.h5")
print('model saved')

model saved
